This is the code that searches for concordance friends of knots.

In [1]:
import regina
import snappy
import time
import itertools
import csv
import pickle
import random
from IPython.display import display, clear_output #used as in-calculation feedback

###### CODE TO ATTACH RIBBON BANDS #######

import snappy
import itertools

#Methods to manipulate knot diagrams

def number_of_strands(PD_code):
    '''
    Returns the number of strands in a diagram.
    '''
    X=[]
    for cros in PD_code:
        X=X+[x for x in cros]
    return max(X)+1 

def get_next_strand(PD_code,strand,crossing):
    '''
    Returns the number of the next strand and its next crossing.
    '''
    PD=PD_code.copy()
    PD.remove(crossing)
    for cros in PD:
        for i in range(4):
            if cros[i]==strand:
                if i==0:
                    y=cros[2]
                if i==1:
                    y=cros[3]
                if i==2:
                    y=cros[0]
                if i==3:
                    y=cros[1]
                return [y,cros]
            
def replace_one_strand_in_crossing(PD_code,cros,old_strand,new_strand):
    '''
    Replaces first occurance of old_strand by new_strand in the crossing. 
    '''
    for i in range(4):
            if cros[i]==old_strand:
                if i==0:
                    new_cros=(new_strand,cros[1],cros[2],cros[3])
                if i==1:
                    new_cros=(cros[0],new_strand,cros[2],cros[3])
                if i==2:
                    new_cros=(cros[0],cros[1],new_strand,cros[3])
                if i==3:
                    new_cros=(cros[0],cros[1],cros[2],new_strand)
                PD_code.remove(cros)
                PD_code.append(new_cros)
                return PD_code

def change_crossing(PD_code,crossing):
    '''
    Changes the crossing.
    '''
    (a,b,c,d)=crossing
    PD_code.remove(crossing)
    PD_code.append((d,a,b,c))
    return PD_code


def strands_in_same_region(PD,strand1,strand2):
    '''Returns True if strand1 and strand2 are in the boundary of the same region of the diagram and False if not.
    We also return a pair of crossings that lie on the same 'side' of the strands 1 and 2, so that we can attach bands.
    WARNING: Might not work if Reidemeister 1 simplifications are posible.'''
    for crossing in PD:
        if strand1 in crossing:
            break
    #crossing contains strand1
    m=number_of_strands(PD)
    
    #Next we run through the knot and always turn to the left and check if we get strand2
    cros=crossing
    strand=strand1
    for i in range(m):
        cros=get_next_strand(PD,strand,cros)[1]
        if strand2==cros[(cros.index(strand)+1)%4]:
            return [True,get_next_strand(PD,strand1,crossing)[1],cros]
        strand=cros[(cros.index(strand)+1)%4]
        if cros==crossing:
            break
    #Next we run through the knot and always turn to the right and check if we get strand2
    cros=crossing
    strand=strand1
    for i in range(m):
        cros=get_next_strand(PD,strand,cros)[1]
        if strand2==cros[(cros.index(strand)-1)%4]:
            return [True,get_next_strand(PD,strand1,crossing)[1],cros]
        strand=cros[(cros.index(strand)-1)%4]
        if cros==crossing:
            break
    return [False,False,False]

#Attach a ribbon band.

def attach_ribbon_band(PD_code,strand1,strand2,twists=0):
    '''Attach a ribbon band between strand1 and strand2 with the specified number of twists.
    Remark: Get the mirror by interchanging the strands.'''
    PD=PD_code.copy()
    [test,cros1,cros2]=strands_in_same_region(PD,strand1,strand2)
    if test==False:
        raise ValueError('The strands do not lie in the same region.')
    m=number_of_strands(PD)
    if cros1!=cros2:
        PD=replace_one_strand_in_crossing(PD,cros1,strand1,m+1)
        PD=replace_one_strand_in_crossing(PD,cros2,strand2,m+7)
    if cros1==cros2:
        PD=replace_one_strand_in_crossing(PD,cros1,strand1,m+1)
        cros2=PD[-1]
        PD=replace_one_strand_in_crossing(PD,cros2,strand2,m+7)
    PD.append((strand1,m+10,m+6,m+11))
    PD.append((m+11,m+6,m+12,m+5))
    PD.append((m+1,m+9,m+2,m+10))
    PD.append((m+9,m+3,m+8,m+2))
    PD.append((m+5,m+12,m+4,strand2))
    PD.append((m+3,m+7,m+4,m+8))
    try:
        snappy.Link(PD)
        if twists>0:
            PD=PD[:-2]
            PD.append((m+5,m+12,m+4,m+2*twists+11))
            PD.append((m+3,m+2*twists+12,m+4,m+8))
            PD.append((strand2,m+13,m+14,m+7))
            for i in range(twists-1):
                PD.append((m+13+2*i,m+13+2*i+2,m+13+2*i+3,m+13+2*i+1)) 
        if twists<0:
            twists=-twists
            PD=PD[:-2]
            PD.append((m+5,m+12,m+4,m+2*twists+11))
            PD.append((m+3,m+2*twists+12,m+4,m+8))
            PD.append((strand2,m+13,m+14,m+7))
            for i in range(twists-1):
                PD.append((m+13+2*i,m+13+2*i+2,m+13+2*i+3,m+13+2*i+1)) 
            for i in range(1,twists):
                PD=change_crossing(PD,PD[-i])
                
    except ValueError:
        PD=PD_code.copy()
        PD=replace_one_strand_in_crossing(PD,cros1,strand1,m+1)
        PD=replace_one_strand_in_crossing(PD,cros2,strand2,m+7)
        PD.append((m+1,m+10,m+6,m+11))
        PD.append((m+11,m+6,m+12,m+5))
        PD.append((strand1,m+9,m+2,m+10))
        PD.append((m+9,m+3,m+8,m+2))
        PD.append((m+3,strand2,m+4,m+8))
        PD.append((m+5,m+12,m+4,m+7))
        if twists>0:
            PD=PD[:-2]
            PD.append((m+5,m+12,m+4,m+2*twists+11))
            PD.append((m+3,m+2*twists+12,m+4,m+8))
            PD.append((m+13,m+14,strand2,m+7))
            for i in range(twists-1):
                PD.append((m+13+2*i,m+13+2*i+2,m+13+2*i+3,m+13+2*i+1)) 
        if twists<0:
            twists=-twists
            PD=PD[:-2]
            PD.append((m+5,m+12,m+4,m+2*twists+11))
            PD.append((m+3,m+2*twists+12,m+4,m+8))
            PD.append((m+7,m+13,m+14,strand2))
            for i in range(twists-1):
                PD.append((m+13+2*i,m+13+2*i+2,m+13+2*i+3,m+13+2*i+1)) 
            for i in range(1,twists):
                PD=change_crossing(PD,PD[-i])
                
    return PD

#The code that creates concordant diagrams.

def concordance_search(PD_code,max_twists=5,bound=False,number_of_tries=100,verbose=False):
    '''Takes as input a diagram with no Reidemeister 1 simplification.
    Returns all possible diagrams obtained by attaching ribbon bands with at most 5 (or any other specified number)
    of twists.
    If the flag bound is set then only a fixed number of random diagrams gets created.'''
    if bound==False:
        PD=PD_code.copy()
        m=number_of_strands(PD)
        pairs = list(itertools.product(list(range(m)),list(range(m))))
        concordant_knots=[]
        for (a,b) in pairs:
            if a!=b:
                for T in range(-max_twists,max_twists+1):
                    try:
                        concordant_knots.append([attach_ribbon_band(PD,a,b,twists=T),a,b,T])
                    except ValueError:
                        pass
        return concordant_knots
    if bound:
        PD=PD_code.copy()
        m=number_of_strands(PD)
        concordant_knots=[]
        for i in range(number_of_tries):
            a=random.randint(0,m-1)
            b=random.randint(0,m-1)
            if a!=b:
                for T in range(-max_twists,max_twists+1):
                    try:
                        concordant_knots.append([attach_ribbon_band(PD,a,b,twists=T),a,b,T])
                    except ValueError:
                        pass
        return concordant_knots

####### SEARCH FOR FRIENDS #######


# Imports
import snappy
import regina
import csv
import sys
from IPython.display import display, clear_output #used as in-calculation feedback

# for timing
import time
# for exponential function
import math
# for random choices
import random

#### Snappy extensions #####

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        try:
            X = M.drill(d)
            X = X.filled_triangulation()
            X.dehn_fill((1,0),-1)
            for i in range(tries):
                if all_positive(X):
                    pos_triangulations.append(X)
                    if len(pos_triangulations)==number:
                        return pos_triangulations
                    break
                X.randomize()
        except (snappy.SnapPeaFatalError,RuntimeError):
            pass

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_is_isometric_to(X,Y,index=50,try_hard=False):
    """
    Returns True if X and Y are isometric.
    Returns False if X and Y appear to be different.
     """ 
    for i in (0,index):
        w=False
        try:
            w=X.is_isometric_to(Y)
            if w==True:
                return w
        except (RuntimeError,snappy.SnapPeaFatalError):
            pass
        X.randomize()
        Y.randomize()
    if try_hard:
        pos_triang_X=find_positive_triangulations(X,number=1,tries=index)
        pos_triang_Y=find_positive_triangulations(Y,number=1,tries=index)
        for X in pos_triang_X:
            for Y in pos_triang_Y:
                w=better_is_isometric_to(X,Y,index=100,try_hard=False)
                if w==True:
                    return w
    return w

# Checking for knot complements

def is_homology_solid_torus(manifold):
    '''
    Checks if the input manifold has homology Z.
    '''
    M=snappy.Manifold(manifold)
    if M.homology().coefficients!=[0]:
        return False
    return True

def is_knot_complement(manifold,index=2):
    '''
    Checks the short fillings to be S3.
    '''
    assert manifold.num_cusps() == 1
    if manifold.homology().elementary_divisors() != [0]:
        return False
    try:
        slopes = manifold.short_slopes(3.5)[0]
    except RuntimeError:
        slopes = []
    (a, b) = manifold.homological_longitude()
    slopes = [s for s in slopes if abs(a*s[1] - b*s[0]) == 1]
    for s in slopes:
        M = snappy.Triangulation(manifold)
        M.dehn_fill(s)
        if is_three_sphere(M,tries=index):
            return True
    return False


def is_three_sphere(manifold,tries=2):
    """
    True means the manifold is definitely S^3.

    False means it is *likely* not S^3.
    """
    T = manifold
    order = T.homology().order()
    if order == 'infinite' or order > 1:
        return False
    G = snappy.Manifold(T)
    if G.solution_type(enum=True) == 1:
         return False
    for i in range(tries):
        if T.fundamental_group().num_generators() == 0:
            return True
        F = T.filled_triangulation()
        if F.fundamental_group().num_generators() == 0:
            return True
        T.randomize()
    return False

def better_length_spectrum(manifold,max_length=5.0,index=10,high_precision=False):
    '''Computes the length spectrum of the input manifold up to the given max_length. 
    Index gives the number of tries and high_precision increases the precision.'''
    M=snappy.Manifold(manifold)
    if high_precision:
        M=M.high_precision()
    for i in range(index):
        try:
            return M.length_spectrum(cutoff=max_length,include_words=True)
        except RuntimeError:
            M.randomize()
    return False

#regina functions

def to_regina(data):
    '''
    This function was written by Dunfield.
    It sends a SnapPy triangulation to regina.
    '''
    if hasattr(data, '_to_string'):
        data = data._to_string()
    if isinstance(data, str):
        if data.find('(') > -1:
            data = closed_isosigs(data)[0]
        return regina.Triangulation3(data)
    assert isinstance(data, regina.Triangulation3)
    return data

def display_check(sig, tri):
    iso_signatures.append(sig)
    if len(iso_signatures)>maximum:
        return True
        #It stops if we have too many signatures
    return False

def drill_edges(T,index=0,verbose=False,cores=1,max_submanifolds=100,max_triangulations=10,randomized=False):
    '''
    Takes as input a regina triangulation and drills out edges and returns a list of its signatures.
    '''
    global iso_signatures
    iso_signatures=[]
    global maximum
    maximum=max_triangulations 
    if index < 0:
        ctr=0
        while len(iso_signatures) < (-1)*index and ctr < 1000:
            ctr+=1
            T = randomise(T, 100, 2/T.size())
            if not T.isoSig() in iso_signatures:
                iso_signatures.append(T.isoSig())
    else:
        T.retriangulate(index,cores,display_check)
    if verbose:
        print('Found',len(iso_signatures),'triangulations')
    drilled=[]
    if randomized==False:
        for sig in iso_signatures:
            Y=regina.Triangulation3(sig)
            for e in range(Y.edges().size()):
                X=regina.Triangulation3(sig)
                X.pinchEdge(X.edge(e))
                if X.homology().isZ():
                    X.intelligentSimplify()
                    drilled.append(X.isoSig())
                    if len(drilled)>max_submanifolds:
                        if verbose:
                            print('Number of submanifolds found:',len(drilled))
                        return drilled
    if randomized==True:
        l=len(iso_signatures)
        for i in range(max_submanifolds):
            sig=iso_signatures[random.randint(0,l-1)]
            Y=regina.Triangulation3(sig)
            for e in range(Y.edges().size()):
                X=regina.Triangulation3(sig)
                X.pinchEdge(X.edge(e))
                if X.homology().isZ():
                    X.intelligentSimplify()
                    drilled.append(X.isoSig())
                    if len(drilled)>max_submanifolds:
                        if verbose:
                            print('Number of submanifolds found:',len(drilled))
                        return drilled
    if verbose:
        print('Number of submanifolds found:',len(drilled))
    return drilled

#####################################################################
#####################################################################
# functions for random walk
#####################################################################
#####################################################################

def choosemove(T, beta):
    x = random.random()
    if x < math.exp((-1)*beta*T.size()):
        a = 1
    else:
        a = 2
    # setup done
    # go up (2-3)
    # most triangles correspond to a valid move, no need to classify
    if a == 1:
        tri = random.choice(range(T.countTriangles()))
        if T.pachner(T.triangle(tri),True,False):
            S = regina.Triangulation3(T)
            S.pachner(S.triangle(tri),False,True)
            return S
    # go down
    elif a == 2:
        # get all possible 3-2 moves
        valid = []
        for e in range(T.countEdges()):
            if T.pachner(T.edge(e),True,False):
                valid.append(e)
        if valid != []:
            S = regina.Triangulation3(T)
            S.pachner(S.edge(random.choice(valid)),False,True)
            return S            
    # nothing worked
    return T

def randomise(T, steps, beta):
    # initialise number of steps
    st = 0
    while st < steps:
        st += 1
        T = choosemove(T,beta)
    return T

#####################################################################
#####################################################################
# end functions for random walk
#####################################################################
#####################################################################



def search_for_friend(knot,effort='low',max_number_of_friends=1,Regina=True,SnapPy_dual_curves=True,SnapPy_length_spectrum=True,MCMC=True,verbose=False):
    '''
    This functions takes as input a triangulation of the exterior of a knot K and searchs for friends of K. 
    (If it is not a knot exterior it still searchs for a knot F on S3 such that 0-surgery on F yields the 
    homological filling on K.)
    
    Method:
    First we create the 0-surgery K(0,1) of K.
    Then we create a list of knot complements in K(0,1). We use the 6-theorem to check if that knot complement 
    has a filling to S3 and thus represents a knot F in S3. If that knot appears to not be isometric to K,
    we return F. 
    WARNING: It is then not rigoursly proven that F and K are different. But that will usually be the case. 
    It needs to be double-checked for example using the volume or another knot invariant.
    
    There are three options to create the list of knot complements in K(0,1).
    
    (1) SnapPy_dual_curves=True: 
    This drills out dual curves of a triangulation of K(0,1)
    (which only works if the triangulation yields a hyperbolic structure).
    
    (2) SnapPy_length_spectrum=True:
    Uses the verified length spectrum computation of K(0,1) and drills out short geodesics from K(0,1).
    
    (3) Regina=True:
    This creates a one-vertex triangulation of K(0,1) in regina and then drills out edges of that triangulation. Uses
    Regina retriangulate function to go through Pachner graph in breadth first search

        
    (4) MCMC=True:
    Same as (3), but the next triangulation is found by a random walk rather than by breadth first search in the Pachner
    graph (as done by "retriangulate").
    
    If Regina is not installed put regina=False. In general it is recommended to use all three methods, since there
    exist friends that can only be found effectively with one of the methods.
    
    
    Options:
    We can put effort='low','medium', 'high' to choose pre-chosen values of parameters to search for friends.
    Or one can put effort=None and choose the parameters freely. See the documentation in the code for that.
    
    
    
    The other options affect the runtime and efficiency of the code:
     
     
     -- 
    How fast the code will run depends of course heavily on the input triangulation. 
    The default options are choosen for maximal efficiency. But if one wants to search harder for friends one 
    should increase the parameters. The following options have turned out to run reasonable fast and find 
    most friends on reasonable small examples.
    '''
    
    
    
    #We have the following parameters:
    #General:
    random=True #If true chooses random dual_curves, geodesics, edges to drill.
    random_upper_bound=100 #Number of random elements that we drill.
    exterior_index=2 #Number of tries to check if a drilled out object is a knot complement.
    try_hard=False #If True it searches very hard for an isometry of a potential friend to K. 
    isometry_index=5 #Number of tries to check if two manifolds are isometric
    
    #For drilling dual curves:
    max_segments=100 #Maximal number of segements in dual curves.
    upper_bound_dual_curves=2000 #An upper bound on the number of dual curves that we check.
    
    #For drilling geodesics
    max_length=5.0 #maximal length of a geodesic in the length spectrum.
    high_precision=False #If true uses high precision to compute the length spectrum.
    recursion_limit=50000 #The recursion limit for drilling out geodesics.
    upper_bound_geodesics=2000 #Upper bound on the number of geodesics we check
    
    #For drilling edges with regina:
    regina_simplify=False #Simplifies the input triangulation if set to True
    regina_retriangulate_index=2 #The maximal difference of the number of simplices of a triangulation from the input triangulation.
    regina_cores_used=1 #If this number gets increased regina uses more cores and the code runs quicker but might also crush your computer.
    max_regina_submanifolds=5000 #The maximal number of submanifolds that regina creates.
    maximal_number_of_Pachner_moves=250 #The maximal number of Pachner moves regina performs to create more triangulations.
    
        
    
    if effort=='low':
        random=False 
        random_upper_bound=100
        exterior_index=1 
        try_hard=False  
        isometry_index=1 
        max_segments=6 
        upper_bound_dual_curves=15 
        max_length=1.0 
        high_precision=False 
        recursion_limit=10000 
        upper_bound_geodesics=15 
        regina_simplify=True
        regina_retriangulate_index=0 
        regina_cores_used=1 
        max_regina_submanifolds=50 
        maximal_number_of_Pachner_moves=10
        rounds = 10
    
    if effort=='medium':
        random=False 
        random_upper_bound=100
        exterior_index=2 
        try_hard=False  
        isometry_index=2 
        max_segments=25 
        upper_bound_dual_curves=100 
        max_length=3.0 
        high_precision=False 
        recursion_limit=40000 
        upper_bound_geodesics=100 
        regina_simplify=True
        regina_retriangulate_index=0 
        regina_cores_used=1 
        max_regina_submanifolds=100 
        maximal_number_of_Pachner_moves=10
        rounds = 30
    
    if effort=='high':
        random=False 
        random_upper_bound=100
        exterior_index=2 
        try_hard=False  
        isometry_index=2 
        max_segments=50 
        upper_bound_dual_curves=250 
        max_length=4.0 
        high_precision=False 
        recursion_limit=50000 
        upper_bound_geodesics=250 
        regina_simplify=True
        regina_retriangulate_index=2 
        regina_cores_used=1 
        max_regina_submanifolds=5000 
        maximal_number_of_Pachner_moves=100
        rounds = 1000
        
    if verbose:
        print('We have choosen the following parameters to search for friends:')
        print('random',random)
        print('random_upper_bound',random_upper_bound)
        print('exterior_index',exterior_index) 
        print('try_hard',try_hard)  
        print('isometry_index',isometry_index) 
        print('max_segments',max_segments) 
        print('upper_bound_dual_curves',upper_bound_dual_curves) 
        print('max_length',max_length) 
        print('high_precision',high_precision) 
        print('recursion_limit',recursion_limit) 
        print('upper_bound_geodesics',upper_bound_geodesics)
        print('regina_simplify',regina_simplify)
        print('regina_retriangulate_index',regina_retriangulate_index)
        print('regina_cores_used',regina_cores_used) 
        print('max_regina_submanifolds',max_regina_submanifolds) 
        print('maximal_number_of_Pachner_moves',maximal_number_of_Pachner_moves)
        print('number of rounds for random walk',rounds)
        print('--------------')

    #M is the knot exterior and K the 0-filling
    M=snappy.Manifold(knot)
    K=snappy.Manifold(knot)
    if is_homology_solid_torus(K)==False:
        raise ValueError('The homology of K is not Z and thus it is not a knot in S3.')        
    K.dehn_fill(K.homological_longitude())
    
    #Lists of possible knot exteriors and friends
    possible_knot_exteriors=[]
    friends=[]
    
    ##### SNAPPY DUAL CURVES ######
    
    if SnapPy_dual_curves:
        dual_curves=K.dual_curves(max_segments=max_segments)
        
        if verbose:
            print('--------------')
            print('We check dual curves with SnapPy and search for a friend.')
            print('Number of dual curves:',len(dual_curves))
            
        if random==False:
            if verbose:
                print('We check all dual curves.')
            for c in dual_curves:
                try:
                    E=K.drill(c)
                    E=E.filled_triangulation()
                    if better_is_isometric_to(E,M,index=isometry_index)==False:
                        if is_homology_solid_torus(E)==True:
                            already_known=False
                            for X in possible_knot_exteriors:
                                if better_is_isometric_to(X,E,index=isometry_index):
                                    already_known=True
                                    break
                            if already_known==False:
                                if verbose:
                                    print('We found a new potential knot exterior:',E)
                                possible_knot_exteriors.append(E)
                                if verbose:
                                    print('Checking for knot complement:',E)
                                if is_knot_complement(E,index=exterior_index):
                                    if better_is_isometric_to(E,M,try_hard=try_hard)==False:
                                        already_known=False
                                        for X in friends:
                                            if better_is_isometric_to(X,E,index=isometry_index,try_hard=try_hard):
                                                already_known=True
                                                break
                                        if already_known==False:
                                            if verbose:
                                                print('We found a friend:',E)
                                            friends.append(E)
                                            if len(friends)>max_number_of_friends-1:
                                                if verbose:
                                                    print('Number of potential knot exteriors checked:',len(possible_knot_exteriors))
                                                    print('Number of friends found:',len(friends))
                                                return friends, dual_curve.index(c)  
                    if c.index>upper_bound_dual_curves:
                        break
                except:
                    pass
                    
        if random==True:
            if verbose:
                print('We check a random collection of dual curves.')
            for i in range(random_upper_bound):
                try:
                    c=dual_curves[random.randint(0,len(dual_curves)-1)]
                    E=K.drill(c)
                    E=E.filled_triangulation()
                    if better_is_isometric_to(E,M,index=isometry_index)==False:
                        if is_homology_solid_torus(E)==True:
                            already_known=False
                            for X in possible_knot_exteriors:
                                if better_is_isometric_to(X,E,index=isometry_index):
                                    already_known=True
                                    break
                            if already_known==False:
                                if verbose:
                                    print('We found a new potential knot exterior:',E)
                                possible_knot_exteriors.append(E)
                                if verbose:
                                    print('Checking for knot complement:',E)
                                if is_knot_complement(E,index=exterior_index):
                                    if better_is_isometric_to(E,M,try_hard=try_hard)==False:
                                        already_known=False
                                        for X in friends:
                                            if better_is_isometric_to(X,E,index=isometry_index,try_hard=try_hard):
                                                already_known=True
                                                break
                                        if already_known==False:
                                            if verbose:
                                                print('We found a friend:',E)
                                            friends.append(E)
                                            if len(friends)>max_number_of_friends-1:
                                                if verbose:
                                                    print('Number of potential knot exteriors checked:',len(possible_knot_exteriors))
                                                    print('Number of friends found:',len(friends))
                                                return friends 
                except:
                    pass
                                        
##### SNAPPY LENGTH SPECTRUM ######
    
    if SnapPy_length_spectrum:
        sys.setrecursionlimit(recursion_limit)
        
        if verbose:
            print('--------------')
            print('We check short geodescis with SnapPy and search for a friend.')
        
        # Compute the length spectrum
        spec=better_length_spectrum(K,max_length=max_length,index=10,high_precision=False)
    
        if spec==False:
            if verbose:
                print('We could not compute the length spectrum of the 0-surgery.')
        else:
            if verbose:
                print('Number of geodesics in the length spectrum:',len(spec))
            
            if random==False:
                if verbose:
                    print('We check all geodesics.')
       
                for c in spec:
                    try:
                        E=K.drill_word(c.word)
                        E=E.filled_triangulation()
                        if better_is_isometric_to(E,M,index=isometry_index)==False:
                            if is_homology_solid_torus(E)==True:
                                already_known=False
                                for X in possible_knot_exteriors:
                                    if better_is_isometric_to(X,E,index=isometry_index):
                                        already_known=True
                                        break
                                if already_known==False:
                                    possible_knot_exteriors.append(E)
                                    if is_knot_complement(E,index=exterior_index):
                                        if better_is_isometric_to(E,M,try_hard=try_hard)==False:
                                            already_known=False
                                            for X in friends:
                                                if better_is_isometric_to(X,E,index=isometry_index,try_hard=try_hard):
                                                    already_known=True
                                                    break
                                            if already_known==False:
                                                if verbose:
                                                    print('We found a friend:',E)
                                                friends.append(E)
                                                if len(friends)>max_number_of_friends-1:
                                                    if verbose:
                                                        print('Number of potential knot exteriors checked:',len(possible_knot_exteriors))
                                                        print('Number of friends found:',len(friends))
                                                    return friends  
                    except:
                        pass
                    if spec.index(c)>upper_bound_geodesics:
                        break
            if random==True:
                if verbose:
                    print('We check a random collection of geodesics.')
                    
                for i in range(random_upper_bound):
                    c=spec[random.randint(0,len(spec)-1)]
                    try:
                        E=K.drill_word(c.word)
                        E=E.filled_triangulation()
                        if better_is_isometric_to(E,M,index=isometry_index)==False:
                            if is_homology_solid_torus(E)==True:
                                already_known=False
                                for X in possible_knot_exteriors:
                                    if better_is_isometric_to(X,E,index=isometry_index):
                                        already_known=True
                                        break
                                if already_known==False:
                                    possible_knot_exteriors.append(E)
                                    if is_knot_complement(E,index=exterior_index):
                                        if better_is_isometric_to(E,M,try_hard=try_hard)==False:
                                            already_known=False
                                            for X in friends:
                                                if better_is_isometric_to(X,E,index=isometry_index,try_hard=try_hard):
                                                    already_known=True
                                                    break
                                            if already_known==False:
                                                if verbose:
                                                    print('We found a friend:',E)
                                                friends.append(E)
                                                if len(friends)>max_number_of_friends-1:
                                                    if verbose:
                                                        print('Number of potential knot exteriors checked:',len(possible_knot_exteriors))
                                                        print('Number of friends found:',len(friends))
                                                    return friends                
                    except:
                        pass
        
    ##### REGIN DRILL EDGES ######                 
    
    if Regina:
        if verbose:
            print('--------------')
            print('We drill out knots via Regina.')
            
        T=to_regina(K.filled_triangulation())
        if regina_simplify:
            T.intelligentSimplify()
            
        if random==False:
            if verbose:
                print('We check all edges.')
            sigs=drill_edges(T,index=regina_retriangulate_index,cores=regina_cores_used,max_submanifolds=max_regina_submanifolds,max_triangulations=maximal_number_of_Pachner_moves)
            if verbose:
                print('Total number of submanifolds found via regina:',len(sigs)) 
                
        if random==True:   
            if verbose:
                print('We check random edges.')
            sigs=drill_edges(T,index=regina_retriangulate_index,cores=regina_cores_used,max_submanifolds=max_regina_submanifolds,max_triangulations=maximal_number_of_Pachner_moves,randomized=True)
            if verbose:
                print('Total number of submanifolds found via regina:',len(sigs)) 

        for sig in sigs:
            try:
                T=regina.Triangulation3(sig)
                E=snappy.Manifold(T.snapPea())
                if better_is_isometric_to(E,M,index=isometry_index)==False:
                    if is_homology_solid_torus(E)==True:
                        already_known=False
                        for X in possible_knot_exteriors:
                            if better_is_isometric_to(X,E):
                                already_known=True
                                break
                        if already_known==False:
                            if verbose:
                                print('We found a new potential knot exterior:',E)
                            possible_knot_exteriors.append(E)
                            if verbose:
                                print('Checking for knot complement:',E)
                            if is_knot_complement(E,index=exterior_index):
                                if better_is_isometric_to(E,M,try_hard=try_hard)==False:
                                    already_known=False
                                    for X in friends:
                                        if better_is_isometric_to(X,E,index=isometry_index,try_hard=try_hard):
                                            already_known=True
                                            break
                                    if already_known==False:
                                        if verbose:
                                            print('We found a friend:',E)
                                        friends.append(E)
                                        if len(friends)>max_number_of_friends-1:
                                            if verbose:
                                                print('Number of potential knot exteriors checked:',len(possible_knot_exteriors))
                                                print('Number of friends found:',len(friends))
                                            return friends 
            except:
                pass

    ##### MCMC to replace retriangulation (more variance for other results -- hopefully) ######                 
    
    if MCMC:        
        if verbose:
            print('--------------')
            print('We drill out knots via Regina, using a random walk method to choose next triangulation.')
            
        T=to_regina(K.filled_triangulation())
        if regina_simplify:
            T.intelligentSimplify()

        if random==False:
            if verbose:
                print('We check all edges.')
            sigs=drill_edges(T,index=(-1)*rounds,cores=regina_cores_used,max_submanifolds=max_regina_submanifolds)
            if verbose:
                print('Total number of submanifolds found via regina:',len(sigs)) 
                
        if random==True:   
            if verbose:
                print('We check random edges.')
            sigs=drill_edges(T,index=(-1)*rounds,cores=regina_cores_used,randomized=True,max_submanifolds=max_regina_submanifolds)
            if verbose:
                print('Total number of submanifolds found via regina:',len(sigs)) 
        ctr=0
        print("found",len(sigs),"triangulations to check")
        for sig in sigs:
            try:
                T=regina.Triangulation3(sig)
                E=snappy.Manifold(T.snapPea())
                if better_is_isometric_to(E,M,index=isometry_index)==False:
                    if is_homology_solid_torus(E)==True:
                        already_known=False
                        for X in possible_knot_exteriors:
                            if better_is_isometric_to(X,E):
                                already_known=True
                                break
                        if already_known==False:
                            if verbose:
                                print('We found a new potential knot exterior:',E)
                            possible_knot_exteriors.append(E)
                            if verbose:
                                print('Checking for knot complement:',E)
                            if is_knot_complement(E,index=exterior_index):
                                if better_is_isometric_to(E,M,try_hard=try_hard)==False:
                                    already_known=False
                                    for X in friends:
                                        if better_is_isometric_to(X,E,index=isometry_index,try_hard=try_hard):
                                            already_known=True
                                            break
                                    if already_known==False:
                                        if verbose:
                                            print('We found a friend:',E)
                                        friends.append(E)
                                        if len(friends)>max_number_of_friends-1:
                                            if verbose:
                                                print('Number of potential knot exteriors checked:',len(possible_knot_exteriors))
                                                print('Number of friends found:',len(friends))
                                            return friends
            except:
                pass


    if verbose:
        print('Number of potential knot exteriors checked:',len(possible_knot_exteriors))
        print('Number of friends found:',len(friends))        
    
    return friends
            
    
def find_S3_filling(manifold,index=2):
    '''
    Checks the short fillings to be S3.
    '''
    
    assert manifold.num_cusps() == 1
    if manifold.homology().elementary_divisors() != [0]:
        return False

    try:
        slopes = manifold.short_slopes(3.5)[0]
    except RuntimeError:
        slopes = []

    (a, b) = manifold.homological_longitude()
    slopes = [s for s in slopes if abs(a*s[1] - b*s[0]) == 1]
    for s in slopes:
        M = snappy.Triangulation(manifold)
        M.dehn_fill(s)
        if is_three_sphere(M,tries=index):
            return s
    return False


            
    
def create_diagram(knot_complement,verbose=False):
    '''
    Takes a list of complements of a knot and creates a diagram.
    '''
    M=knot_complement
    M.dehn_fill((0,0))
    D=False
    try:
        M.set_peripheral_curves('shortest')
    except:
        pass
    try:
        M.dehn_fill((1,0))
        if is_three_sphere(M):
            D=M.exterior_to_link()
            if D!=False:
                D.simplify('global')
                PD_friend=D.PD_code()
                return PD_friend
            
        M.dehn_fill((0,0))
        M.dehn_fill(find_S3_filling(M))
        D=M.exterior_to_link()
        if D!=False:
            D.simplify('global')
            PD_friend=D.PD_code()
            return PD_friend
    except:
        pass

    print('COULD NOT FIND A DIAGRAM!')
    return D
        
        
###### CONCORDANCE FRIEND SEARCH ########

def concordance_friend_search(D,number_of_concordance_friends=1,effort='low',max_twists=1,number_of_tries=100,verbose=False):
    '''
    Input is a knot diagram D of a knot K.
    Warning: The diagram D should not allow any Reidemeister 1 simplifications.
    Search for a concordance friend of first order, i.e. a knot F that is a friend with a knot C concordant to K.
    Possible values for effort are 'low', 'medium' and 'high'.
    Or 'None' if the parameters are choosen directly.
    '''
    
    #Parameters
    #max_twists= Number of twists of ribbon bands.
    #number_of_tries= Maximal number of ribbon bands tried
    bound=True #If false, tries all possible ribbon bands.
    
    
    
    if effort=='low':
        SnapPy_length_spectrum=False
    
    if effort=='medium': 
        SnapPy_length_spectrum=False
    
    if effort=='high':
        SnapPy_length_spectrum=True
        
        
    
    PD=D.PD_code()
    
    conc_diagrams=concordance_search(PD,max_twists=max_twists,bound=bound,number_of_tries=number_of_tries)
    
    conc=[]
    for c in conc_diagrams:
        C=snappy.Link(c[0]).exterior()
        already_known=False
        for X in [y[0] for y in conc]:
            if better_is_isometric_to(X,C,index=2):
                already_known=True
                break
        if already_known==False:
            conc.append([C,c])
                
    
    if verbose:
        print('Number of concordant knots created:',len(conc))
        
    concordance_friends=[]
    for c in conc:
        try:
            C=c[0]
            F=search_for_friend(C,effort=effort,max_number_of_friends=number_of_concordance_friends,SnapPy_length_spectrum=SnapPy_length_spectrum)
            if F!=[]:
                if verbose:
                    print('The concordant knot:',c,'has friends.')
                    print('Number of friends:',len(F))
                for f in F:
                    concordance_friends.append([f,c])
                    
            if len(concordance_friends)>number_of_concordance_friends-1:
                if verbose:
                    print('Total number of concordance friends found:',len(concordance_friends))
                return concordance_friends
        except:
            pass
    
    if verbose:
        print('Number of concordance friends found:',len(concordance_friends))
    return concordance_friends


##### RIBBON SEARCH CODE ######

import ribbon.rw  # the main class
import ribbon.visualizer as visualizer  # for visualizing the output

import numpy as np  # handles arrays
import snappy  # to represent the knots
import logging  # to print some info of what the code is doing

def is_ribbon(PD,tries=10000,twists=5):
    '''Takes as input a PD code and checks if it is ribbon.
    If True is returned it is proven to be ribbon. If False is returned it is not clear.'''

    links = [PD]

    # links to search. Accepts a list of PD codes or link names. K6a3 is the Stevedore knot
    max_size = len(PD)+25          # max number of crossings any intermediate knot can have
    max_steps =  int(tries/10)         # max number of steps searched by the random walker before giving up and resetting the link
    max_tries = tries         # number of total steps before we give up completely
    max_bct = twists               # we use the same variable to set the max number of allowed twists, link components, and bands to add. If you start with a knot, the max number of bands is the max number of components + 1, since each band adds a link component
    log_level = logging.ERROR # controls how much information is printed by the code while searching for a band
    use_checks = False        # If set to true (whcih requires sage), the code will check for slice obstructions after attaching bands, rather than keep searching more and more bands on a link that is potentially obstructed. This uses the Fox Milnor condition, which requires the Alexander Polynomial. This can be slow to compute for large knots
    save_images = True        # Will use the visualizer to save the band found by the random walker

    random_walker = ribbon.rw.RandomWalker(links=links, 
                                           max_size=max_size, 
                                           max_steps=max_steps, 
                                           max_bct=max_bct, 
                                           logger=None, 
                                           log_level=log_level, 
                                           use_band_checks=use_checks, 
                                           save_solved_knot_images=save_images
                                          )

    tries = 0
    while tries < max_tries:
        tries += 1
        if tries % 10000 == 0:
            print("Performed {:d} steps.".format(tries))

        # Find all valid actions
        valid_actions = np.argwhere(random_walker.invalid_action_mask()).flatten()

        # pick a random one
        a = np.random.choice(valid_actions) if len(valid_actions) > 0 else 0

        # perform the action
        done, info = random_walker.step(a)

        # check whether the knot is done (either because the unknot was reached, or a reset was triggered)
        if done:
            if 'unknot' in info['result']:  # found bands
                print("Knot is ribbon!")  # this is not an error, we just want to print this message irrespective of the logger level.
                return True
    return False


Some demonstration. We start with some interesting knots:

In [2]:
#Simplest slice knot
S=snappy.Manifold('K6a3') 

#Random slice knot
R=snappy.Manifold('K12n870') 

#Conway knot, top slice but not smoothly slice
C=snappy.Manifold('K11n34') 

#The positive Whiteheaddouble of the right-handed trefoil, top slice but not smoothly slice
W_p=snappy.Link([(31, 25, 32, 24), (26, 36, 27, 35), (36, 30, 37, 29), (30, 15, 31, 16), (22, 18, 23, 17), (16, 24, 17, 23), (13, 34, 14, 35), (37, 10, 0, 11), (25, 15, 26, 14), (7, 33, 8, 32), (9, 19, 10, 18), (8, 21, 9, 22), (27, 4, 28, 5), (12, 6, 13, 5), (33, 7, 34, 6), (3, 28, 4, 29), (2, 12, 3, 11), (1, 21, 2, 20),(19, 1, 20, 0)]).exterior()

#The negative Whiteheaddouble of the right-handed trefoil, top slice unknown if smoothly slice
W_n=snappy.Link([(31, 39, 32, 38), (36, 30, 37, 29), (24, 36, 25, 35), (32, 3, 33, 4), (33, 9, 34, 8), (11, 31, 12, 30), (5, 28, 6, 29), (27, 6, 28, 7), (26, 16, 27, 15), (16, 26, 17, 25), (9, 3, 10, 2), (10, 39, 11, 0), (37, 12, 38, 13), (4, 14, 5, 13), (23, 19, 24, 18), (19, 23, 20, 22), (17, 34, 18, 35), (14, 8, 15, 7), (1, 20, 2, 21), (21, 0, 22, 1)]).exterior()

#The figure eigth knot
E=snappy.Manifold('K4a1') 

#The (2,1)-cable of the figure eigth knot, top slice, but not smoothly slice
def braid_index(word):
    '''Returns the braid index of a word'''
    return max([abs(x) for x in word])+1

def writhe(word):
    '''Returns the writhe of a braid word.'''
    wr=0
    for w in word:
        wr=wr+sign(w)
    return wr
    
def cable(word,p,q):
    '''Returns a braid word of the cable. (For positive p and arbitrary q.)'''
    cable_word=[]
    for i in word:
        subword=[]
        for t in range(0,p):
            subword=subword+list(range(p*abs(i)+t,p*(abs(i)-1)+t,-1))
        if i<0:
            subword=[-j for j in subword]
        cable_word=cable_word+subword
    wr=writhe(word)
    if (q-p*wr)<0:
        cable_word=cable_word+(p*wr-q)*list(range(-1,-(p-1)-1,-1))
    if (q-p*wr)>=0:
        cable_word=cable_word+(-p*wr+q)*list(range(1,(p-1)+1,+1))
    return cable_word

CE=snappy.Link(braid_closure=cable(E.link().braid_word(),2,1)).exterior()

In [3]:
gui tk

We first verify that the slice knots are slice, and check if they have friends.

In [4]:
is_ribbon(S.link().PD_code())

Knot is ribbon!


True

In [5]:
is_ribbon(R.link().PD_code())

Knot is ribbon!


True

In [6]:
search_for_friend(S)

[]

In [7]:
search_for_friend(R)

[K12n870-9_filled(0,0)]

Next we check if we can find the friends of the Conway knot and check if the other interesting knots have friends.

In [8]:
search_for_friend(C)

[Regina_Triangulation(0,0)]

In [9]:
search_for_friend(W_p)

[]

In [10]:
search_for_friend(W_n)

[]

In [11]:
search_for_friend(CE)

[]

Next we search for concordance friends of the non-hyperbolic examples.

The negative Whitehead double of the right-handed trefoil.

In [17]:
concordance_friends=concordance_friend_search(W_n.link(),number_of_concordance_friends=10,effort='low',max_twists=5,number_of_tries=100,verbose=True)


Number of concordant knots created: 151
The concordant knot: [unnamed link(0,0), [[(21, 0, 22, 1), (1, 20, 2, 21), (9, 3, 10, 2), (32, 3, 33, 4), (4, 14, 5, 13), (14, 8, 15, 7), (33, 9, 34, 8), (10, 39, 11, 0), (11, 31, 12, 30), (37, 12, 38, 13), (26, 16, 27, 15), (16, 26, 17, 25), (17, 34, 18, 35), (23, 19, 24, 18), (19, 23, 20, 22), (24, 36, 25, 35), (36, 30, 37, 29), (31, 39, 32, 38), (41, 28, 6, 29), (27, 6, 28, 47), (41, 50, 46, 51), (51, 46, 52, 45), (5, 49, 42, 50), (49, 43, 48, 42), (45, 52, 44, 61), (43, 62, 44, 48), (47, 53, 54, 7), (60, 59, 61, 62), (58, 57, 59, 60), (56, 55, 57, 58), (54, 53, 55, 56)], 5, 7, -5]] has friends.
Number of friends: 1
Number of concordance friends found: 1


In [18]:
F=concordance_friends[0][0]
D=create_diagram(F)
D

[(159, 99, 160, 98),
 (371, 252, 372, 253),
 (392, 570, 393, 569),
 (477, 562, 478, 563),
 (389, 101, 390, 100),
 (478, 91, 479, 92),
 (391, 251, 392, 250),
 (359, 560, 360, 561),
 (388, 338, 389, 337),
 (298, 506, 299, 505),
 (602, 503, 603, 504),
 (195, 502, 196, 503),
 (62, 502, 63, 501),
 (530, 500, 531, 499),
 (299, 37, 300, 36),
 (600, 33, 601, 34),
 (193, 34, 194, 35),
 (63, 33, 64, 32),
 (532, 32, 533, 31),
 (533, 632, 534, 633),
 (65, 630, 66, 631),
 (191, 629, 192, 628),
 (598, 630, 599, 629),
 (300, 627, 301, 628),
 (529, 454, 530, 455),
 (61, 452, 62, 453),
 (196, 452, 197, 451),
 (603, 451, 604, 450),
 (296, 449, 297, 450),
 (534, 268, 535, 267),
 (526, 105, 527, 106),
 (57, 104, 58, 105),
 (200, 104, 201, 103),
 (607, 103, 608, 102),
 (292, 101, 293, 102),
 (294, 156, 295, 155),
 (605, 154, 606, 155),
 (198, 153, 199, 154),
 (60, 152, 61, 151),
 (263, 145, 264, 144),
 (531, 149, 532, 148),
 (229, 142, 230, 143),
 (453, 150, 454, 151),
 (634, 145, 635, 146),
 (29, 147, 30,

In [19]:
len(D)

322

In [20]:
is_ribbon(D)

Performed 10000 steps.


False

The positive Whitehead double of the right-handed trefoil.

In [24]:
concordance_friends=concordance_friend_search(W_p.link(),number_of_concordance_friends=1,effort='low',max_twists=6,number_of_tries=100,verbose=True)


Number of concordant knots created: 133
The concordant knot: [unnamed link(0,0), [[(19, 1, 20, 0), (1, 21, 2, 20), (3, 28, 4, 29), (27, 4, 28, 5), (33, 7, 34, 6), (7, 33, 8, 32), (8, 21, 9, 22), (9, 19, 10, 18), (37, 10, 0, 11), (13, 34, 14, 35), (25, 15, 26, 14), (30, 15, 31, 16), (16, 24, 17, 23), (22, 18, 23, 17), (31, 25, 32, 24), (26, 36, 27, 35), (36, 30, 37, 29), (12, 6, 13, 39), (2, 12, 45, 11), (39, 48, 44, 49), (49, 44, 50, 43), (5, 47, 40, 48), (47, 41, 46, 40), (43, 50, 42, 59), (41, 60, 42, 46), (45, 51, 52, 3), (58, 57, 59, 60), (56, 55, 57, 58), (54, 53, 55, 56), (52, 51, 53, 54)], 5, 3, -5]] has friends.
Number of friends: 1
Total number of concordance friends found: 1


In [25]:
F=concordance_friends[0][0]
D=create_diagram(F)
D

[(4, 170, 5, 169),
 (522, 460, 523, 459),
 (1, 282, 2, 283),
 (523, 255, 0, 254),
 (0, 88, 1, 87),
 (497, 174, 498, 175),
 (495, 35, 496, 34),
 (493, 10, 494, 11),
 (484, 461, 485, 462),
 (482, 278, 483, 277),
 (480, 259, 481, 260),
 (479, 109, 480, 108),
 (475, 96, 476, 97),
 (46, 486, 47, 485),
 (384, 471, 385, 472),
 (183, 473, 184, 472),
 (356, 474, 357, 473),
 (250, 456, 251, 455),
 (453, 249, 454, 248),
 (45, 460, 46, 461),
 (446, 174, 447, 173),
 (448, 35, 449, 36),
 (450, 10, 451, 9),
 (125, 444, 126, 445),
 (440, 131, 441, 132),
 (133, 438, 134, 439),
 (436, 303, 437, 304),
 (309, 428, 310, 429),
 (427, 227, 428, 226),
 (66, 432, 67, 431),
 (424, 315, 425, 316),
 (385, 269, 386, 268),
 (349, 391, 350, 390),
 (380, 188, 381, 187),
 (270, 380, 271, 379),
 (383, 101, 384, 100),
 (271, 360, 272, 361),
 (357, 98, 358, 99),
 (355, 266, 356, 267),
 (352, 179, 353, 180),
 (310, 227, 311, 228),
 (68, 305, 69, 306),
 (257, 278, 258, 279),
 (286, 247, 287, 248),
 (42, 282, 43, 281),
 (26

In [26]:
len(D)

262

In [27]:
is_ribbon(D)

Performed 10000 steps.


False

The (2,1)-cable of the figure eight knot.

In [8]:
concordance_friends=concordance_friend_search(CE.link(),number_of_concordance_friends=1,effort='low',max_twists=1,number_of_tries=100,verbose=True)


Number of concordant knots created: 32
The concordant knot: [unnamed link(0,0), [[(17, 1, 18, 0), (1, 9, 2, 8), (2, 26, 3, 25), (14, 3, 15, 4), (31, 4, 32, 5), (5, 28, 6, 29), (6, 11, 7, 12), (18, 8, 19, 7), (9, 17, 10, 16), (10, 0, 11, 33), (13, 20, 14, 21), (26, 16, 27, 15), (30, 21, 31, 22), (22, 29, 23, 30), (27, 33, 28, 32), (23, 35, 24, 13), (41, 25, 20, 24), (35, 44, 40, 45), (45, 40, 46, 39), (12, 43, 36, 44), (43, 37, 42, 36), (37, 19, 38, 42), (39, 46, 38, 41)], 12, 19, 0]] has friends.
Number of friends: 1
Total number of concordance friends found: 1


In [11]:
F=concordance_friends[0][0]
D=create_diagram(F)
D

[(26, 161, 27, 162),
 (145, 99, 146, 98),
 (129, 164, 130, 165),
 (8, 166, 9, 165),
 (137, 33, 138, 32),
 (197, 31, 198, 30),
 (69, 80, 70, 81),
 (194, 89, 195, 90),
 (91, 158, 92, 159),
 (92, 30, 93, 29),
 (88, 109, 89, 110),
 (67, 186, 68, 187),
 (75, 180, 76, 181),
 (76, 39, 77, 40),
 (61, 41, 62, 40),
 (62, 182, 63, 181),
 (60, 143, 61, 144),
 (79, 147, 80, 146),
 (124, 177, 125, 178),
 (128, 150, 129, 149),
 (125, 36, 126, 37),
 (196, 157, 197, 158),
 (134, 153, 135, 154),
 (68, 58, 69, 57),
 (183, 143, 184, 142),
 (182, 41, 183, 42),
 (184, 202, 185, 201),
 (126, 208, 127, 207),
 (59, 202, 60, 203),
 (77, 205, 78, 204),
 (185, 97, 186, 96),
 (127, 103, 128, 102),
 (58, 97, 59, 98),
 (78, 100, 79, 99),
 (144, 204, 145, 203),
 (90, 215, 91, 216),
 (123, 226, 124, 227),
 (195, 214, 196, 215),
 (136, 211, 137, 212),
 (107, 155, 108, 154),
 (193, 110, 194, 111),
 (135, 106, 136, 107),
 (13, 179, 14, 178),
 (11, 206, 12, 207),
 (10, 101, 11, 102),
 (9, 148, 10, 149),
 (55, 6, 56, 7),
 

In [12]:
len(D)

114

In [15]:
is_ribbon(D)

Performed 10000 steps.


False

Finally, we check the most promising example, the Conway knot. We create some concordance friends and check if any of them is ribbon.

In [4]:
concordance_friends=concordance_friend_search(C.link(),number_of_concordance_friends=100,effort='low',max_twists=6,number_of_tries=250,verbose=True)


Number of concordant knots created: 271
The concordant knot: [unnamed link(0,0), [[(3, 1, 4, 0), (1, 7, 2, 6), (11, 4, 12, 5), (8, 16, 9, 15), (20, 9, 21, 10), (10, 17, 11, 18), (18, 13, 19, 14), (14, 19, 15, 20), (16, 0, 17, 21), (7, 3, 8, 23), (5, 12, 29, 13), (2, 32, 28, 33), (33, 28, 34, 27), (23, 31, 24, 32), (31, 25, 30, 24), (27, 34, 26, 39), (25, 40, 26, 30), (6, 35, 36, 29), (35, 37, 38, 36), (37, 39, 40, 38)], 2, 6, 3]] has friends.
Number of friends: 1
The concordant knot: [unnamed link(0,0), [[(3, 1, 4, 0), (1, 7, 2, 6), (7, 3, 8, 2), (8, 16, 9, 15), (20, 9, 21, 10), (10, 17, 11, 18), (18, 13, 19, 14), (14, 19, 15, 20), (16, 0, 17, 21), (11, 23, 12, 5), (5, 12, 29, 13), (23, 32, 28, 33), (33, 28, 34, 27), (4, 31, 24, 32), (31, 25, 30, 24), (27, 34, 26, 43), (25, 44, 26, 30), (29, 35, 36, 6), (42, 41, 43, 44), (40, 39, 41, 42), (38, 37, 39, 40), (36, 35, 37, 38)], 4, 6, -5]] has friends.
Number of friends: 1
The concordant knot: [unnamed link(0,0), [[(3, 1, 4, 0), (1, 7, 2, 

In [5]:
diagrams=[]

for x in concordance_friends:
    F=x[0]
    D=create_diagram(F)
    diagrams.append([D,x])
    print('Number of crossings of concordance friend:',len(D))
    print('Is ribbon:',is_ribbon(D))
    
diagrams

Number of crossings of concordance friend: 202
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 125
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 134
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 166
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 191
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 135
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 382
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 124
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 140
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 102
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 102
Performed 10000 steps.
Is ribbon: False
Number of crossings of concordance friend: 

[[[(160, 46, 161, 45),
   (72, 213, 73, 214),
   (91, 0, 92, 1),
   (82, 9, 83, 10),
   (403, 92, 0, 93),
   (137, 215, 138, 214),
   (175, 221, 176, 220),
   (332, 207, 333, 208),
   (261, 208, 262, 209),
   (357, 212, 358, 213),
   (259, 363, 260, 362),
   (206, 265, 207, 266),
   (234, 268, 235, 267),
   (186, 264, 187, 263),
   (333, 265, 334, 264),
   (70, 258, 71, 257),
   (210, 361, 211, 362),
   (232, 366, 233, 365),
   (182, 360, 183, 359),
   (331, 365, 332, 364),
   (71, 357, 72, 356),
   (177, 216, 178, 217),
   (235, 204, 236, 205),
   (304, 128, 305, 127),
   (306, 369, 307, 370),
   (305, 270, 306, 271),
   (354, 68, 355, 67),
   (255, 69, 256, 68),
   (143, 62, 144, 63),
   (224, 63, 225, 64),
   (358, 182, 359, 181),
   (262, 186, 263, 185),
   (135, 180, 136, 181),
   (335, 189, 336, 188),
   (74, 180, 75, 179),
   (271, 126, 272, 127),
   (278, 38, 279, 37),
   (218, 29, 219, 30),
   (256, 114, 257, 113),
   (355, 113, 356, 112),
   (138, 111, 139, 112),
   (69, 115,